<a href="https://colab.research.google.com/github/frankx1/deepspeech_project/blob/main/DeepSpeech_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Train our own model**
- setting up environment
- install modules and create directories
- the checkpoint files have to be uploaded manually since they are too big to be stored remotely


In [1]:
!pip install gitpython
!pip install virtualenv
import git
import os

git.Repo.clone_from('https://github.com/mozilla/DeepSpeech','DeepSpeech')

     |████████████████████████████████| 163kB 7.9MB/s 
     |████████████████████████████████| 71kB 6.6MB/s 
     |████████████████████████████████| 5.7MB 7.9MB/s 
     |████████████████████████████████| 337kB 57.9MB/s 


<git.repo.base.Repo '/content/DeepSpeech/.git'>

In [2]:
!apt-get install python3-venv
!python3 -m venv /content/deepspeech-train-venv/
!source /content/deepspeech-train-venv/bin/activate

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  python-pip-whl python3.6-venv
The following NEW packages will be installed:
  python-pip-whl python3-venv python3.6-venv
0 upgraded, 3 newly installed, 0 to remove and 14 not upgraded.
Need to get 1,660 kB of archives.
After this operation, 1,902 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 python-pip-whl all 9.0.1-2.3~ubuntu1.18.04.4 [1,653 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 python3.6-venv amd64 3.6.9-1~18.04ubuntu1.3 [6,180 B]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 python3-venv amd64 3.6.7-1~18.04 [1,208 B]
Fetched 1,660 kB in 1s (1,187 kB/s)
Selecting previously unselected package python-pip-whl.
(Reading database ... 144865 files and directories currently installed.)
Preparing to unpack .../python-pip-

In [3]:
os.chdir('DeepSpeech')
!pip3 install --upgrade pip==20.2.2 wheel==0.34.2 setuptools==49.6.0
!pip3 install --upgrade -e .

     |████████████████████████████████| 1.5MB 7.6MB/s 
     |████████████████████████████████| 808kB 35.7MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
  Found existing installation: wheel 0.35.1
    Uninstalling wheel-0.35.1:
      Successfully uninstalled wheel-0.35.1
  Found existing installation: setuptools 50.3.2
    Uninstalling setuptools-50.3.2:
      Successfully uninstalled setuptools-50.3.2


Obtaining file:///content/DeepSpeech
     |████████████████████████████████| 49 kB 3.6 MB/s 
     |████████████████████████████████| 258 kB 11.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.7 MB 15.3 MB/s 
     |████████████████████████████████| 2.1 MB 24.1 MB/s 
     |████████████████████████████████| 110.5 MB 24 kB/s 
     |████████████████████████████████| 159 kB 83.5 MB/s 
     |████████████████████████████████| 81 kB 11.6 MB/s 
     |████████████████████████████████| 50 kB 8.2 MB/s 
     |████████████████████████████████| 3.8 MB 68.7 MB/s 
     |████████████████████████████████| 503 kB 64.5 MB/s 
     |████████████████████████████████| 75 kB 5.6 MB/s 
     |████████████████████████████████| 106 kB 81.4 MB/s 
     |████████████████████████████████| 49 kB 7.3 MB/s 
     |████████████████████████████████| 133 kB 77.4 MB/s 
  Created wheel for opuslib: filen

In [4]:
!sudo apt-get install python3-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-dev is already the newest version (3.6.7-1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


In [5]:
!pip3 uninstall tensorflow
!pip3 install 'tensorflow-gpu==1.15.4'

Found existing installation: tensorflow 1.15.4
Uninstalling tensorflow-1.15.4:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/freeze_graph
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-1.15.4.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-1.15.4
     |████████████████████████████████| 411.0 MB 16 kB/s 


In [6]:
!make Dockerfile.train

sed \
	-e "s|#DEEPSPEECH_REPO#|https://github.com/mozilla/DeepSpeech.git|g" \
	-e "s|#DEEPSPEECH_SHA#|origin/master|g" \
	< Dockerfile.train.tmpl > Dockerfile.train


In [7]:
git.Repo.clone_from('https://github.com/frankx1/deepspeech_project','audioset', branch = 'audioset')

<git.repo.base.Repo '/content/DeepSpeech/audioset/.git'>

In [8]:
os.mkdir('fine_tuning_checkpoints')

# **Fine-tuning the model**
- Since the orginally provided model in DeepSpeech website cannot recognize audio with noises, our target is to make the model work with noise. In order to achieve our goal, we use the"fine-tuning" method listed in the DeepSpeech documentation. The idea is to start from a pretrained model and train it using our audio with noise.
- The model is from the DeepSpeech website(deepspeech-0.9.2)
- We 
- Since our training data is fairly small, we also use them as testing data
- the training epoch has been set to 200, with 0.0001 learning rate
- After 200 epochs, the loss is pretty low and it seems like the matching rate is high. However, this only applies to the audios that we've already trained. The model is not generalized with the specific type of noise. We believe that is because of our sample size.

In [24]:
!python3 DeepSpeech.py --n_hidden 2048 --checkpoint_dir fine_tuning_checkpoints --epochs 200 --train_files audioset/my-train.csv --dev_files audioset/my-dev.csv --test_files audioset/my-test.csv --learning_rate 0.0001

Traceback (most recent call last):
  File "DeepSpeech.py", line 7, in <module>
    from deepspeech_training import train as ds_train
  File "/content/DeepSpeech/training/deepspeech_training/train.py", line 16, in <module>
    import tensorflow as tf
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/__init__.py", line 102, in <module>
    from tensorflow_core import *
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/__init__.py", line 28, in <module>
    from tensorflow.python import pywrap_tensorflow  # pylint: disable=unused-import
  File "<frozen importlib._bootstrap>", line 1007, in _handle_fromlist
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/__init__.py", line 50, in __getattr__
    module = self._load()
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/__init__.py", line 44, in _load
    module = _importlib.import_module(self.__name__)
  File "/usr/lib/python3.6/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gc